In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = "loan_status"

In [3]:
df = pd.read_csv('LoanStats_2019Q1.csv', skiprows=1)

/Users/gretapfundt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.loc[:, columns].copy()

In [5]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()

In [6]:
# Remove the `Issued` loan status
df = df.loc[df['loan_status'] != 'Issued']

In [7]:
# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

df = pd.get_dummies(df,columns = ['home_ownership','pymnt_plan','initial_list_status',
                                  'application_type','debt_settlement_flag','hardship_flag',
                                  'next_pymnt_d','issue_d','verification_status'])

In [8]:
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag_N,hardship_flag_N,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
93,10500.0,0.1719,375.35,66000.0,Current,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,0,1,0,1,0
99,25000.0,0.2000,929.09,105000.0,Current,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,0,1,0,0,1
132,20000.0,0.2000,529.88,56000.0,Current,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,0,1,0,0,1
133,10000.0,0.1640,353.55,92000.0,Current,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,0,1,0,0,1
140,22000.0,0.1474,520.39,52000.0,Current,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,0,1,1,0,0


In [9]:
# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 
                  'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag_N,hardship_flag_N,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,0,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,0,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,0,1,1,0,0


In [10]:
#create featurs
y = df[target].replace({'low_risk':1,'high_risk':0})
X = df.drop(columns = [target])

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 95 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            68817 non-null  float64
 1   int_rate                             68817 non-null  float64
 2   installment                          68817 non-null  float64
 3   annual_inc                           68817 non-null  float64
 4   dti                                  68817 non-null  float64
 5   delinq_2yrs                          68817 non-null  float64
 6   inq_last_6mths                       68817 non-null  float64
 7   open_acc                             68817 non-null  float64
 8   pub_rec                              68817 non-null  float64
 9   revol_bal                            68817 non-null  float64
 10  total_acc                            68817 non-null  float64
 11  out_prncp                   

In [12]:
# Check the balance of our y values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [13]:
#test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')